# Génération de réponses originales

In [1]:
pip install --upgrade tensorflow

     |████████████████████████████████| 421.8MB 38kB/s 
     |████████████████████████████████| 3.8MB 52.3MB/s 
     |████████████████████████████████| 450kB 63.8MB/s 
     |████████████████████████████████| 81kB 12.7MB/s 
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstal

Modèle

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!ls "/content/gdrive"

'My Drive'


In [0]:
import pandas as pd

data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/echantillon.txt', header = None, sep='\t')

Setup - Import TensorFlow and other libraries

In [0]:
#conda install tensorflow

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time

In [0]:
text = open('/content/gdrive/My Drive/Colab Notebooks/echantillon.txt', 'rb').read().decode(encoding='utf-8')

In [6]:
print ('Length of text: {} characters'.format(len(text)))

Length of text: 67402508 characters


In [7]:
print(text[:250])

Intéressant, ton choix de mots. Putain, elle est la seule chose qui les intéresse. Remets ton cache-nez ! Il faudra lui parler, Mme Kane. Plus fort, blanc-bec. C'était un très beau match. Vous avez raison. Tu dois apprendre comment... Mon scrotum Ce 


In [8]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))


348 unique characters


Process the text
Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
import pickle

with open('/content/gdrive/My Drive/Colab Notebooks/char2idx.pkl','wb') as f:
    pickle.dump(char2idx, f)

with open('/content/gdrive/My Drive/Colab Notebooks/idx2char.pkl','wb') as f:
    pickle.dump(idx2char, f)

In [11]:
text_as_int

array([37, 74, 80, ..., 72, 65, 79])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to len(unique).

In [12]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')


{
  ' ' :   0,
  '!' :   1,
  '"' :   2,
  '#' :   3,
  '$' :   4,
  '%' :   5,
  "'" :   6,
  '(' :   7,
  ')' :   8,
  '*' :   9,
  '+' :  10,
  ',' :  11,
  '-' :  12,
  '.' :  13,
  '/' :  14,
  '0' :  15,
  '1' :  16,
  '2' :  17,
  '3' :  18,
  '4' :  19,
  ...
}


In [13]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))


'Intéressant, ' ---- characters mapped to int ---- > [ 37  74  80 183  78  65  79  79  61  74  80  11   0]


The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?
Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [14]:
char2idx

{' ': 0,
 '!': 1,
 '"': 2,
 '#': 3,
 '$': 4,
 '%': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 '*': 9,
 '+': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 ';': 26,
 '=': 27,
 '@': 28,
 'A': 29,
 'B': 30,
 'C': 31,
 'D': 32,
 'E': 33,
 'F': 34,
 'G': 35,
 'H': 36,
 'I': 37,
 'J': 38,
 'K': 39,
 'L': 40,
 'M': 41,
 'N': 42,
 'O': 43,
 'P': 44,
 'Q': 45,
 'R': 46,
 'S': 47,
 'T': 48,
 'U': 49,
 'V': 50,
 'W': 51,
 'X': 52,
 'Y': 53,
 'Z': 54,
 '[': 55,
 '\\': 56,
 ']': 57,
 '^': 58,
 '_': 59,
 '`': 60,
 'a': 61,
 'b': 62,
 'c': 63,
 'd': 64,
 'e': 65,
 'f': 66,
 'g': 67,
 'h': 68,
 'i': 69,
 'j': 70,
 'k': 71,
 'l': 72,
 'm': 73,
 'n': 74,
 'o': 75,
 'p': 76,
 'q': 77,
 'r': 78,
 's': 79,
 't': 80,
 'u': 81,
 'v': 82,
 'w': 83,
 'x': 84,
 'y': 85,
 'z': 86,
 '{': 87,
 '}': 88,
 '~': 89,
 '\x80': 90,
 '\x81': 91,
 '\x83': 92,
 '\x86': 93,
 '\x87': 94,
 '\x88': 95,
 '\x89': 96,
 '\x8a': 97,
 '\x8

In [15]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])


I
n
t
é
r


The batch method lets us easily convert these individual characters to sequences of the desired size.

In [16]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))


'Intéressant, ton choix de mots. Putain, elle est la seule chose qui les intéresse. Remets ton cache-n'
"ez ! Il faudra lui parler, Mme Kane. Plus fort, blanc-bec. C'était un très beau match. Vous avez rais"
"on. Tu dois apprendre comment... Mon scrotum Ce n'est pas très sur. Alors, vous étiez amis toi et le "
"gargarisme. Abandonnez. Inutile, me voici. Bon, peut-être que je n'aurais pas dû en parler maintenant"
". Ce n'était pas mon intention. Non, je ne parle pas ukrainien, mais je parle enveloppe de cash. Je p"


For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


In [18]:
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

Print the first examples input and target values:

In [19]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))


Input data:  'Intéressant, ton choix de mots. Putain, elle est la seule chose qui les intéresse. Remets ton cache-'
Target data: 'ntéressant, ton choix de mots. Putain, elle est la seule chose qui les intéresse. Remets ton cache-n'


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the RNN considers the previous step context in addition to the current input character.

In [20]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))


Step    0
  input: 37 ('I')
  expected output: 74 ('n')
Step    1
  input: 74 ('n')
  expected output: 80 ('t')
Step    2
  input: 80 ('t')
  expected output: 183 ('é')
Step    3
  input: 183 ('é')
  expected output: 78 ('r')
Step    4
  input: 78 ('r')
  expected output: 65 ('e')


Create training batches

We used tf.data to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [21]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

Build The Model

Use tf.keras.Sequential to define the model. For this simple example three layers are used to define our model:

    tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions;
    tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use a LSTM layer here.)
    tf.keras.layers.Dense: The output layer, with vocab_size outputs.


In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024


In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)


For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

A drawing of the data passing through the model
Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [25]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 100, 348) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is 100 but the model can be run on inputs of any length:

In [26]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           89088     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 348)           356700    
Total params: 4,384,092
Trainable params: 4,384,092
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.
Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()


This gives us, at each timestep, a prediction of the next character index:

In [28]:
sampled_indices

array([312, 192, 261, 116, 198,  20, 345,  35, 264, 136, 293, 274, 131,
       150, 131, 197, 205,  51, 322, 148, 143, 255, 100,  13, 231,  56,
        18, 338, 312, 196, 206, 149, 192, 174,  73,   7,  62, 247, 102,
        43, 171, 294, 134, 142, 111, 138,  48,  86,  23,  46, 242,  67,
       311, 338,  22,   3,  70,  61,  33, 107, 198,  68, 259,  66, 201,
        96, 250, 310, 188, 209, 277,   2,  42, 335,  86, 193, 271, 153,
       296, 320, 329,  85,  46, 157, 235, 128, 336, 258, 342, 118, 125,
        97, 262, 334,   5, 161, 294,  80, 179, 274])

Decode these to see the text predicted by this untrained model:

In [29]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))


Input: 
 "est là. Comme si j'étais un politicien. une sorte de monarque. {\\pos(192,210)}S'il préfère se vautre"

Next Char Predictions: 
 '〠òτ\xa0ù5ﬀGЈ¶\u200eк°Å°øĂW㨰Ã¾θ\x8e.ű\\3獶〠öăÄòàm(bΜ\x90OÛ–³½\x9a¸Tz8RΒg《獶7#jaE\x96ùhξfü\x89Τ⼼îčс"N牯zóзÈ‚㤊晦yRÌƒ\xad猠ν眾¢©\x8aϋ潦%Ñ–tåк'


Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.
Attach an optimizer, and a loss function

The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the from_logits flag.

In [30]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())


Prediction shape:  (64, 100, 348)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.8529687


Configure the training procedure using the tf.keras.Model.compile method. We'll use tf.keras.optimizers.Adam with default arguments and the loss function.

In [0]:
model.compile(optimizer='adam', loss=loss)


Configure checkpoints
Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = '/content/gdrive/My Drive/Colab Notebooks/training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [0]:
EPOCHS=10

In [34]:
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [35]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 10427 steps
Epoch 1/10
10427/10427 [==============================] - 648s 62ms/step - loss: 1.2733
Epoch 2/10
10427/10427 [==============================] - 666s 64ms/step - loss: 1.1645
Epoch 3/10
10427/10427 [==============================] - 672s 64ms/step - loss: 1.1654
Epoch 4/10
10427/10427 [==============================] - 678s 65ms/step - loss: 1.3552
Epoch 5/10
10427/10427 [==============================] - 686s 66ms/step - loss: 1.8118
Epoch 6/10
10427/10427 [==============================] - 664s 64ms/step - loss: 1.4334
Epoch 7/10
10427/10427 [==============================] - 651s 62ms/step - loss: 1.2792
Epoch 8/10
10427/10427 [==============================] - 660s 63ms/step - loss: 1.2440
Epoch 9/10
10427/10427 [==============================] - 657s 63ms/step - loss: 1.2332
Epoch 10/10
10427/10427 [==============================] - 652s 63ms/step - loss: 1.2330


In [0]:
model_json = model.to_json()
with open("/content/gdrive/My Drive/Colab Notebooks/model_generation_text.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("/content/gdrive/My Drive/Colab Notebooks/model_generation_text.h5")

Generate text
Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different batch_size, we need to rebuild the model and restore the weights from the checkpoint.

In [37]:
tf.train.latest_checkpoint(checkpoint_dir)

'/content/gdrive/My Drive/Colab Notebooks/training_checkpoints/ckpt_10'

In [0]:
model_generation_text = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model_generation_text.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model_generation_text.build(tf.TensorShape([1, None]))


In [39]:
model_generation_text.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            89088     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 348)            356700    
Total params: 4,384,092
Trainable params: 4,384,092
Non-trainable params: 0
_________________________________________________________________


The prediction loop

The following code block generates the text:

    It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

    Get the prediction distribution of the next character using the start string and the RNN state.

    Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

    The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.

To generate text the model's output is fed back to the input

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [0]:
def generate_text(model_generation_text, start_string, num_generate = 100):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model_generation_text(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))


In [45]:
print(generate_text(model_generation_text, start_string=u"ok"))

okue. Ravissave te voir. C'est Merci. Jusqu'à tes marines. Tu ne gâchis pas de dire: La pigroticulé re


In [42]:
len(text)

67402508